# Exploring Hacker News Data

Hacker News is a new incubator site that runs in a similar fashion to Reddit. Users can submit posts and the community votes on whether they enjoy it or not. It is very popular in the technology and computer circle. The data we will be looking at today comes from Kaggle (https://www.kaggle.com/hacker-news/hacker-news-posts) and has been reduced in size from 300,000 posts to remove posts that did not receive any community interaction and then further reduced using a random sampling of 20,000.

In this analysis, I want to take a look at two specific types of posts: the Ask Hacker News (Ask HN) and Show Hacker News (Show HN). Ask HN posts are just that, someone asks a question and the community provides answers. Show HN posts are typically just a way for a user to show an interesting web page, project, etc. to the community.

The information I want to learn is
1. Do Ask HN or Show HN posts receive more comments?
2. Is there an ideal time to post in order to receive the most comments?

With that, let's get started.

In [14]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

In [15]:
print(hn[:6])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Here I've imported the Hacker News data and displayed the first five rows to include the header row. I'm going to print the header row below so we have a reference going forward. Then I will remove the header row to simplify the analysis

In [16]:
headers = hn[0]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [17]:
hn = hn[1:]

In [18]:
print(hn[:3])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']]


I printed the first three rows just to confirm that the header has been removed. Everything looks good. Now we need to separate out the data for Ask HN and Show HN for analysis.

In [19]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    #Finds posts in the Ask HN category
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    #Finds Posts in the Show HN category
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    #Collects all other posts
    else:
        other_posts.append(row)

print('The number of Ask HN posts is ', len(ask_posts))
print('The number of Show HN posts is ', len(show_posts))
print('The number of other posts is ', len(other_posts))

The number of Ask HN posts is  1744
The number of Show HN posts is  1162
The number of other posts is  17194


So here we can see that both Ask HN and Show HN posts together make up roughly 20% of all posts on Hacker News. While Ask HN has the numbers advantage in number of posts, we will need to do a little more analysis to find which of the two gets more comments.

In [20]:
total_ask_comments = 0 #Initial variable to store the number of Ask HN comments
for row in ask_posts:
    ask_comments = int(row[4]) #num_comments is the fifth column in ask_posts
    total_ask_comments += ask_comments #adds posts comments to total

avg_ask_comments = total_ask_comments / len(ask_posts)#divivdes total omments by number of posts
print(avg_ask_comments)

total_show_comments = 0 #Initial variable to store the number of Show HN comments
for row in show_posts:
    show_comments = int(row[4]) #num_comments is the fifth column in show_posts
    total_show_comments += show_comments #adds posts comments to total

avg_show_comments = total_show_comments / len(show_posts)#divivdes total omments by number of posts
print(avg_show_comments)

14.038417431192661
10.31669535283993


Here we can see that not only are Ask HN posts more numerous by count, they also attract more comments than Show HN posts. It is likely that this is simply due to the style of post. Show HN posts just show information whereas Ask HN posts ask a question and begin a conversation.

Our first question has been answered; Ask posts receive more comments than Show posts. Next we will look to answer our second question: Is there an ideal time to post to receive more comments? To do this, we will look at our Ask posts.

In [21]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6] #post time
    comments = int(row[4]) #number of comments, conerted from string to integer
    result_list.append([created_at,comments])
    
counts_by_hour = {} #posts for each hour
comments_by_hour = {} #comments each hour
for row in result_list:
    time = row[0]
    comment = row[1]
    hour = dt.datetime.strptime(time,'%m/%d/%Y %H:%M').strftime('%H')
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comment
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comment
    
    

In [22]:
avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour, (comments_by_hour[hour]/counts_by_hour[hour])])
    
for hour in avg_by_hour:
    print(hour)

['13', 14.741176470588234]
['02', 23.810344827586206]
['17', 11.46]
['08', 10.25]
['20', 21.525]
['22', 6.746478873239437]
['05', 10.08695652173913]
['18', 13.20183486238532]
['14', 13.233644859813085]
['09', 5.5777777777777775]
['15', 38.5948275862069]
['16', 16.796296296296298]
['21', 16.009174311926607]
['01', 11.383333333333333]
['12', 9.41095890410959]
['00', 8.127272727272727]
['04', 7.170212765957447]
['19', 10.8]
['06', 9.022727272727273]
['11', 11.051724137931034]
['10', 13.440677966101696]
['23', 7.985294117647059]
['07', 7.852941176470588]
['03', 7.796296296296297]


Our results are in, however they are a bit hard to read. I'll format this list to display the timestamps in chronological order.

In [31]:
swap_avg_by_hour = [] #swapping hour and average comments to sort by avgerage comments later
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse = True)#sorting by average comments
print('Top 5 Hours for Ask Posts Comments')

for avg, hr in sorted_swap[:5]:#select from top 5 by average comments
    print(
    '{}:00 {:.2f} average comments per post'.format(hr, avg)
    )

Top 5 Hours for Ask Posts Comments
15:00 38.59 average comments per post
02:00 23.81 average comments per post
20:00 21.52 average comments per post
16:00 16.80 average comments per post
21:00 16.01 average comments per post


We have finally answered both questions. Our first being that Ask HN posts receive more comments on average than Show HN posts and secondly, the best time to post.

Looking at the top 5 post time by average comments, we can see that the most comments are coming in between 3PM and 2AM. Two stretches of time appear to be the best, those being between 3PM and 5PM and then 8PM to 10PM. We can go as far as to guess that these post times suggest that users are engaging on Hacker News at the end of a traditional 9-5 work shift and right after dinner time.

So, if you want to rack up some comments on HAcker News, post Ask Hacker News at the end of work and right after dinner. 